In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-10-17"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10106,2024-10-17,Islândia Premier League,16:15,Tindastoll,Haukar,1.11,6.15,179.5,1.83,1.83,-13.5,2.00,1.72,lIKPtI4k,0.900901,0.162602,0.546448,0.546448,0.063503,194.038,76.916753,0.396400,1.2,1.643168,1.369306,132.06,2.120,0.965324,0.455341,-3.0,206.522,52.259656,0.253046,0.6,1.341641,2.236068,268.00,2.392,0.600766,0.251156,-33.0,93,80,1.42,3.35,121.800,231.426,0.981768,0.000000,0.106446,-2.92,-0.584,-0.188356,0.710553,0.4,-0.310553,-3.42,-0.684,-7.529240,0.000000,0.0,0.000000
10107,2024-10-17,Islândia Premier League,16:15,Stjarnan,IR Reykjavik,1.04,9.90,177.5,1.83,1.83,-18.5,2.00,1.72,AX8YvdZ1,0.961538,0.101010,0.546448,0.546448,0.062549,155.586,52.043207,0.334498,2.4,1.341641,0.559017,129.63,1.782,0.651091,0.365371,19.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,87,82,1.49,2.57,147.020,0.000,1.145332,0.000000,0.106446,0.03,0.006,6.666667,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10108,2024-10-17,Islândia Premier League,16:15,Alftanes,Valur,1.77,1.91,175.5,1.83,1.83,-2.5,2.01,1.72,b74S0wdL,0.564972,0.523560,0.546448,0.546448,0.088532,189.162,68.435234,0.361781,0.6,1.341641,2.236068,132.00,2.236,0.677591,0.303037,-28.0,163.390,50.662292,0.310070,1.2,1.643168,1.369306,120.56,1.968,0.580620,0.295031,12.0,80,88,1.65,1.37,137.726,186.462,0.053802,0.000000,0.109952,-0.27,-0.054,-14.259259,0.000000,0.0,0.000000,-2.53,-0.506,-1.798419,0.560518,0.5,-0.060518
10109,2024-10-17,Europa Eurocopa Feminina,15:30,Namur F,San Giovanni F,3.26,1.27,131.5,1.83,1.83,6.5,2.02,1.67,hzkfcs45,0.306748,0.787402,0.546448,0.546448,0.094150,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,52,49,1.46,4.77,0.000,0.000,0.621255,0.000000,0.134139,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10110,2024-10-17,Europa Eurocopa Feminina,15:30,Lattes Montpellier F,Caledonia Gladiators F,1.02,18.00,142.5,1.84,1.82,-24.5,1.92,1.75,rTRbmiIj,0.980392,0.055556,0.543478,0.549451,0.035948,117.864,41.339208,0.350737,1.8,1.643168,0.912871,75.48,1.654,0.775841,0.469070,35.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,74,62,1.02,2.10,0.000,0.000,1.262531,0.007728,0.065509,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10111,2024-10-17,Europa Eurocopa Feminina,14:00,CSM Constanta F,Keltern F,1.16,4.31,140.5,1.80,1.86,-14.5,2.01,1.68,rVd8j324,0.862069,0.232019,0.555556,0.537634,0.094088,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,95,63,1.20,14.70,0.000,0.000,0.814401,0.023184,0.126474,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10112,2024-10-17,Europa Eurocopa Feminina,14:00,Besiktas F,Benfica F,1.01,11.70,138.5,1.88,1.79,-25.5,1.97,1.71,8YHNebvp,0.990099,0.085470,0.531915,0.558659,0.075569,192.172,117.603033,0.611968,1.8,1.643168,0.912871,71.19,2.980,2.121356,0.711864,-10.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,63,77,1.13,2.47,113.278,0.000,1.189453,0.034681,0.099917,1.10,0.220,0.045455,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.00

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
10121,12:00,Lituânia Nkl,Zalgiris Kaunas 2,Silute,1.54,Back Home
10136,19:30,Brasil Nbb,Bauru,Minas,2.37,Back Home
